In [1]:
!pip install numpy
!pip install pandas
!pip install transformers


In [2]:
import numpy as np 
import pandas as pd 


df = pd.read_csv('datasetV2.csv')
df.head()

,ID,Summary,Reviews per Month
0,16976743,Charming yet Trendy Retreat- in my 2 bedroom w...,1.02
1,6556848,Grab your friends and make this two bedroom Br...,0.44
2,240918,A private bedroom in a shared apartment. One b...,1.96
3,4759718,"BEFORE BOOKING: PRIVATE rental only, June to A...",0.25
4,2591213,Beautiful private room on the garden/basement...,2.33


In [3]:

#df = df.sample(frac=0.1, random_state=42)

In [4]:
df

,ID,Summary,Reviews per Month
0,16976743,Charming yet Trendy Retreat- in my 2 bedroom w...,1.02
1,6556848,Grab your friends and make this two bedroom Br...,0.44
2,240918,A private bedroom in a shared apartment. One b...,1.96
3,4759718,"BEFORE BOOKING: PRIVATE rental only, June to A...",0.25
4,2591213,Beautiful private room on the garden/basement...,2.33
...,...,...,...
15449,6967723,"PERFECT FOR YOUNG, EDGY PEOPLE LOOKING FOR AN ...",1.77
15450,165461,NaN,0.69
15451,15799340,Private STUDIO w/private kitchen in one bedroo...,5.14
15452,3053443,My home is in the very center of San Francisco...,1.82


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import BertPreTrainedModel, BertModel, BertTokenizer

from sklearn import metrics
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import torch

'''
# Check if GPU is available
if torch.cuda.is_available():
    # Set the default tensor type to CUDA tensor
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.set_default_device(device)'''


/home/bob/anaconda3/envs/newenvn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'\n# Check if GPU is available\nif torch.cuda.is_available():\n    # Set the default tensor type to CUDA tensor\n    torch.set_default_tensor_type(torch.cuda.FloatTensor)\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n\ntorch.set_default_device(device)'

In [6]:
import os
os.environ['TOKENIZERS_PARALLELISM']= "false"

In [7]:
df['SummaryClean']= df['Summary'].map(lambda x : str(x))

In [8]:
len(df.SummaryClean.tolist())

15454

In [9]:
from transformers import AutoConfig
from transformers import BertTokenizer, BertModel

MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

encoded_corpus = tokenizer(text=df.SummaryClean.tolist(),
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=200,
                            return_attention_mask=True)
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

In [10]:
labels = df['Reviews per Month'].map(lambda x: float(x))

In [11]:
from sklearn.model_selection import train_test_split
test_size = 0.1
seed = 42
train_inputs, test_inputs, train_labels, test_labels = \
            train_test_split(input_ids, labels, test_size=test_size, 
                             random_state=seed)
train_masks, test_masks, _, _ = train_test_split(attention_mask, 
                                        labels, test_size=test_size, 
                                        random_state=seed)

In [12]:
from sklearn.preprocessing import StandardScaler
price_scaler = StandardScaler()
price_scaler.fit(train_labels.to_numpy().reshape(-1, 1))
train_labels = price_scaler.transform(train_labels.to_numpy().reshape(-1, 1))
test_labels = price_scaler.transform(test_labels.to_numpy().reshape(-1, 1))

In [13]:
import torch
from torch.utils.data import TensorDataset, DataLoader
batch_size = 16
def create_dataloaders(inputs, masks, labels, batch_size):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks)
    labels_tensor = torch.tensor(labels)
    dataset = TensorDataset(input_tensor, mask_tensor, 
                            labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, 
                            shuffle=False)
    return dataloader
train_dataloader = create_dataloaders(train_inputs, train_masks, 
                                      train_labels, batch_size)
test_dataloader = create_dataloaders(test_inputs, test_masks, 
                                     test_labels, 1)

In [14]:
import torch.nn as nn
from transformers import BertModel
class BertRegressor(nn.Module):
    
    def __init__(self, drop_rate=0.2, freeze_bert=True):
        super(BertRegressor, self).__init__()
        D_in, D_out = 768, 1
        self.bert = \
                   BertModel.from_pretrained(MODEL_NAME)
        '''for param in self.bert.parameters():
            param.requires_grad = False'''
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, 128),
            nn.ReLU(),
            nn.Dropout(drop_rate),
            nn.Linear(128, D_out)
            )
        self.double()
    def forward(self, input_ids, attention_masks):
        
        outputs = self.bert(input_ids, attention_masks)
        class_label_output = outputs[1]
        outputs = self.regressor(class_label_output)
        return outputs
model = BertRegressor(drop_rate=0.2)

In [15]:
#model = torch.load("trained_model")

In [16]:
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
model.to(device)

Using GPU.


BertRegressor(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [17]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr=5e-5,
                  eps=1e-8)

/home/bob/anaconda3/envs/newenvn/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
from transformers import get_linear_schedule_with_warmup
epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,       
                 num_warmup_steps=0, num_training_steps=total_steps)

In [19]:
loss_function = nn.MSELoss()


In [20]:
def predict(model, dataloader, device):
    model.eval()
    output = []
    labels = []
    for batch in dataloader:
        batch_inputs, batch_masks, label = \
                                  tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, 
                            batch_masks)
            labels += label
    return output, labels

In [21]:
def evaluate1():
        test_dataloader2 = create_dataloaders(test_inputs, test_masks, 
                                test_labels, 1)
        y_pred_scaled2, labels2 = predict(model, test_dataloader2, device)
        y_test2= list(map(lambda x : x.to("cpu").numpy()[0], labels2))
        y_pred2= list(map(lambda x : x.to("cpu").numpy()[0], y_pred_scaled2))
        from sklearn.metrics import mean_absolute_error
        from sklearn.metrics import median_absolute_error
        from sklearn.metrics import mean_squared_error
        from sklearn.metrics import mean_absolute_percentage_error
        from sklearn.metrics import r2_score
        mae = mean_absolute_error(y_test2, y_pred2)
        mdae = median_absolute_error(y_test2, y_pred2)
        mse = mean_squared_error(y_test2, y_pred2)
        mape = mean_absolute_percentage_error(y_test2, y_pred2)
        #mdape = ((pd.Series(y_test) - pd.Series(y_pred))\
                # / pd.Series(y_test)).abs().median()
        r_squared = r2_score(y_test2, y_pred2)
        print(mae, mdae, mse, mape, r_squared)

In [22]:
def evaluate2():
        test_dataloader2 = create_dataloaders(train_inputs, train_masks, 
                                train_labels, 1)
        y_pred_scaled2, labels2 = predict(model, test_dataloader2, device)
        y_test2= list(map(lambda x : x.to("cpu").numpy()[0], labels2))
        y_pred2= list(map(lambda x : x.to("cpu").numpy()[0], y_pred_scaled2))
        from sklearn.metrics import mean_absolute_error
        from sklearn.metrics import median_absolute_error
        from sklearn.metrics import mean_squared_error
        from sklearn.metrics import mean_absolute_percentage_error
        from sklearn.metrics import r2_score
        mae = mean_absolute_error(y_test2, y_pred2)
        mdae = median_absolute_error(y_test2, y_pred2)
        mse = mean_squared_error(y_test2, y_pred2)
        mape = mean_absolute_percentage_error(y_test2, y_pred2)
        #mdape = ((pd.Series(y_test) - pd.Series(y_pred))\
                # / pd.Series(y_test)).abs().median()
        r_squared = r2_score(y_test2, y_pred2)
        print(mae, mdae, mse, mape, r_squared)

In [23]:

def r2_score(outputs, labels):
    labels_mean = torch.mean(labels)
    ss_tot = torch.sum((labels - labels_mean) ** 2)
    ss_res = torch.sum((labels - outputs) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [24]:
#evaluate2()

In [25]:
from torch.nn.utils import clip_grad_norm_
def train(model, optimizer, scheduler, loss_function, epochs,       
          train_dataloader, device, clip_value=2):
    for epoch in range(epochs):
        print(epoch)
        print("-----")
        model.train()
        for step, batch in enumerate(train_dataloader): 
            print(step)  
            batch_inputs, batch_masks, batch_labels = \
                               tuple(b.to(device) for b in batch)
            model.zero_grad()
            outputs = model(batch_inputs, batch_masks)     
                
            loss = (loss_function(outputs.squeeze(), 
                             batch_labels.squeeze()))
            loss.backward()
            clip_grad_norm_(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()
        evaluate1()
        evaluate1()
                
    return model
model = train(model, optimizer, scheduler, loss_function, epochs, 
              train_dataloader, device, clip_value=2)

0
-----
0


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
#y_pred_scaled, labels = predict(model, test_dataloader, device)

In [ ]:
'''y_test= labels
y_pred = y_pred_scaled'''

In [ ]:
#y_test= list(map(lambda x : x.to("cpu").numpy()[0], labels))


In [ ]:
#y_pred = list(map(lambda x : x.to("cpu").numpy()[0], y_pred_scaled))

In [ ]:
#y_pred

[0.023064011837874085,
 -0.10991329677906922,
 -0.05402781242897528,
 0.05508394453169933,
 -0.17078740765062764,
 -0.20052938036584114,
 -0.2323236041217319,
 -0.20790779139990276,
 0.10572542415406282,
 -0.15537824670527986,
 0.03081146681085495,
 -0.09967908904861547,
 -0.10491911290311268,
 0.06310059502342544,
 -0.04634752524509919,
 -0.14868289646757366,
 0.061789579514265704,
 -0.16430775794658306,
 0.005901065482177531,
 -0.026196783438810864,
 -0.06426587381796706,
 0.07697857556329785,
 -0.07307595831961775,
 -0.038953498535157005,
 0.128182429764538,
 -0.03129068158921114,
 -0.22323336197761343,
 0.05877904084934935,
 -0.0330344880219416,
 0.15945316672067938,
 0.14725612563745025,
 -0.15699464177144004,
 -0.12674356450853716,
 -0.060681201303760146,
 -0.01585626681583395,
 -0.10636627396045303,
 0.09331280344864536,
 -0.16628836912672965,
 0.08664041216296564,
 -0.1840831352660747,
 -0.048489791173552815,
 0.18276843020326106,
 -0.012601398392319996,
 0.05891683060622872,
 

In [ ]:
#y_test

[tensor([-0.4619], device='cuda:0', dtype=torch.float64),
 tensor([-0.9083], device='cuda:0', dtype=torch.float64),
 tensor([-0.0434], device='cuda:0', dtype=torch.float64),
 tensor([-0.8804], device='cuda:0', dtype=torch.float64),
 tensor([-0.9585], device='cuda:0', dtype=torch.float64),
 tensor([-0.8134], device='cuda:0', dtype=torch.float64),
 tensor([-0.8413], device='cuda:0', dtype=torch.float64),
 tensor([-0.7743], device='cuda:0', dtype=torch.float64),
 tensor([-0.7688], device='cuda:0', dtype=torch.float64),
 tensor([-0.9585], device='cuda:0', dtype=torch.float64),
 tensor([0.0682], device='cuda:0', dtype=torch.float64),
 tensor([0.4923], device='cuda:0', dtype=torch.float64),
 tensor([-0.6125], device='cuda:0', dtype=torch.float64),
 tensor([3.4107], device='cuda:0', dtype=torch.float64),
 tensor([-0.0713], device='cuda:0', dtype=torch.float64),
 tensor([-0.5623], device='cuda:0', dtype=torch.float64),
 tensor([1.2456], device='cuda:0', dtype=torch.float64),
 tensor([-0.5567],

In [ ]:
#y_pred = price_scaler.inverse_transform(np.array(y_pred_scaled).reshape(-1,1))

In [ ]:
'''from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
mae = mean_absolute_error(y_test, y_pred)
mdae = median_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
#mdape = ((pd.Series(y_test) - pd.Series(y_pred))\
        # / pd.Series(y_test)).abs().median()
r_squared = r2_score(y_test, y_pred)'''

In [ ]:
#mae, mdae, mse, mape, r_squared

(0.7280485022849993,
 0.6463390034491812,
 0.9119851236004479,
 1.0946573377981599,
 0.08801487639955208)

In [ ]:
'''def evaluate():
        test_dataloader2 = create_dataloaders(test_inputs, test_masks, 
                                test_labels, 1)
        y_pred_scaled2, labels2 = predict(model, test_dataloader2, device)
        y_test2= list(map(lambda x : x.to("cpu").numpy()[0], labels2))
        y_pred2= list(map(lambda x : x.to("cpu").numpy()[0], y_pred_scaled2))
        from sklearn.metrics import mean_absolute_error
        from sklearn.metrics import median_absolute_error
        from sklearn.metrics import mean_squared_error
        from sklearn.metrics import mean_absolute_percentage_error
        from sklearn.metrics import r2_score
        mae = mean_absolute_error(y_test2, y_pred2)
        mdae = median_absolute_error(y_test2, y_pred2)
        mse = mean_squared_error(y_test2, y_pred2)
        mape = mean_absolute_percentage_error(y_test2, y_pred2)
        #mdape = ((pd.Series(y_test) - pd.Series(y_pred))\
                # / pd.Series(y_test)).abs().median()
        r_squared = r2_score(y_test2, y_pred2)
        print(mae, mdae, mse, mape, r_squared)'''

In [ ]:
'''from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
mae = mean_absolute_error(y_test2, y_pred2)
mdae = median_absolute_error(y_test2, y_pred2)
mse = mean_squared_error(y_test2, y_pred2)
mape = mean_absolute_percentage_error(y_test2, y_pred2)
#mdape = ((pd.Series(y_test) - pd.Series(y_pred))\
        # / pd.Series(y_test)).abs().median()
r_squared = r2_score(y_test2, y_pred2)'''

In [ ]:
#mae, mdae, mse, mape, r_squared

(0.7338061747313142,
 0.6844490938418113,
 0.8452176128890638,
 1.1206721551252143,
 0.019483549335861827)

In [ ]:
#torch.save(model,"trained_model")